In [16]:
# 필요한 모델들을 import 합니다.

# 기본
import os
from tqdm import tqdm

# 이미지 처리
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(42)


# torch Data
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# model 
import torch
import torch.nn as nn
import torch.optim as optim

#dataloader
from tools.SignalDataSet import SignalDataSet

# wandb
import wandb

In [17]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [18]:
root_dir = "Data/data2/"
train_dataset = SignalDataSet(root_dir+"train")
val_dataset = SignalDataSet(root_dir+"val")
test_dataset = SignalDataSet(root_dir+"test")

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1,16,5), # 244 -> 240
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 240 -> 120
            nn.Conv2d(16,32,5), # 120 -> 116
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 116 -> 58
            nn.Conv2d(32,64,5), # 58 -> 54
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 54 -> 27
        )

        self.fc = nn.Sequential(
            nn.Linear(36864, 27*27),
            nn.ReLU(),
            nn.Linear(27*27, 100),
            nn.ReLU(),
            nn.Linear(100, 7),
        )

    def forward(self, x):
        out = self.conv_layer(x)
        out = out.view(out.size(0), -1) # 참고 view는 reshape과 역할 비슷
        out = self.fc(out)
        return out

In [20]:
loss_func = nn.CrossEntropyLoss()

In [21]:
# 모델 생성
model = CNN().double().to(DEVICE)

#파라미터 설정
EPOCHS = 20
BATCH_SIZE = 4
lr = 0.00001

optimizer = optim.Adam(model.parameters(), lr=lr)

#wandb
config={
    "architecture": "CNN-custom",
    "dataset": "custom",

    "batch_size": BATCH_SIZE,
    "learning_rate": BATCH_SIZE,
    "epochs": EPOCHS,
    }
wandb.init(project="drone_classification", config=config)
wandb.run.name = 'CNN-custom1'

# dataloader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [22]:
loss_arr = []

for i in range(EPOCHS):
    best_model = None
    best_acc = 0

    # train
    print(f"Epochs {i+1}")
    train_loss_sum = 0
    for [image,label] in train_loader:
        x = image.double().to(DEVICE)
        y = label.to(DEVICE)

        output = model.forward(x)
        _,output_max = torch.max(output,1)

        loss = loss_func(output,y)
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.cpu().detach().numpy()
    print(f"train loss = {loss}")

    #val
    val_loss_sum = 0
    with torch.no_grad():
        total = 0
        correct = 0

        for [image,label] in val_loader:
            x = image.to(DEVICE)
            y = label.to(DEVICE)

            output = model.forward(x)
            _,output_max = torch.max(output,1)

            loss = loss_func(output,y)
            val_loss_sum += loss.cpu().detach().numpy()

            total += label.size(0)
            correct += (output_max == torch.argmax(y,1)).sum().cpu().detach().numpy()
            acc = correct / total

            if acc > best_acc:
                best_acc = acc
                best_model = model

    wandb.log({"train_loss": train_loss_sum, "val_loss": val_loss_sum, "val_acc": acc})

    print(f"val loss = {loss}")
    print(f"val acc = {acc}")
    print("---------------------------------")

wandb.finish()

Epochs 1
train loss = 0.336051631856909
val loss = 0.4832284432726211
val acc = 0.6585365853658537
---------------------------------
Epochs 2
train loss = 0.09259654007622134
val loss = 1.1659860309814851
val acc = 0.725609756097561
---------------------------------
Epochs 3
train loss = 0.03933024472158356
val loss = 0.001589364827648287
val acc = 0.774390243902439
---------------------------------
Epochs 4
train loss = 0.0001062679828171872
val loss = 0.10273391546318507
val acc = 0.8048780487804879
---------------------------------
Epochs 5
train loss = 4.6748420867591856e-05
val loss = 0.07592364970826973
val acc = 0.7804878048780488
---------------------------------
Epochs 6


In [ ]:
# test data 생성
test_size = 100
test_data = make_circle(val_size) + make_triangle(val_size) + make_rectangle(val_size)
test_label = [0] * val_size + [1] * val_size + [2] * val_size

test_dataset = test_dataset
test_loader = DataLoader(test_dataset, batch_size=3*test_size, shuffle=False)

# 모델에서 backprop을 하지 않도록 설정
best_model.eval()

# test
first_batch = next(iter(test_loader))
result = best_model(first_batch[0].to(DEVICE))
test_correct = (torch.argmax(result,1).cpu().detach().numpy() == torch.argmax(first_batch[1],1).numpy()).sum()
test_acc = test_correct / len(test_data)

print(f"test acc = {test_acc}") 